In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import gc
gc.collect()

0

In [2]:
data=pd.read_csv('./data/data.csv')

In [3]:
# data.head()

In [4]:
print(data.shape)
print(data[data['label']==-1].shape)
print(data.label.value_counts())

(2099998, 5)
(50000, 5)
 0    1287218
 1     762780
-1      50000
Name: label, dtype: int64


In [5]:
data['query_len']=data['query'].apply(lambda x:len(x.replace('{','').replace('}','').split(',')))
# print(data[data['label']!=-1].groupby('query_len')['label'].mean())
# print('====================')
data['prefix_len']=data['prefix'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('prefix_len')['label'].mean())
# print('====================')
data['tag_len']=data['tag'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('tag_len')['label'].mean())
# print('====================')
data['title_len']=data['title'].apply(lambda x:len(str(x)))
# print(data[data['label']!=-1].groupby('title_len')['label'].mean())
# print('====================')
# print(data.shape)
# print('====================')
# print(data.columns)
# print('====================')
# print(data.describe())
# print('====================')
# print(data.info())

In [6]:
gc.collect()

28

In [7]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
lr_col=['prefix','title','tag','query']
for feature in lr_col:
    data[feature]=data[feature].astype(str)

data['prefix']=le.fit_transform(data['prefix'])
data['title']=le.fit_transform(data['title'])
data['query']=le.fit_transform(data['query'])
data['tag']=le.fit_transform(data['tag'])

In [8]:
#对应不同特征的点击率的构造
train_data=data[['prefix','query','title','tag','query_len','prefix_len','tag_len','title_len','label']]
col=['prefix','query','title','tag','query_len','prefix_len','tag_len','title_len']
for item in col:
    temp=train_data.groupby(item,as_index=False)['label'].agg({item+'_click':'sum',item+'_count':'count'})
    temp[item+'_ctr']=(temp[item+'_click']/temp[item+'_count'])
    train_data = pd.merge(train_data, temp, on=item, how='left')
print(train_data.shape)

(2099998, 33)


In [9]:
cols=['prefix','query','title','tag']
for i in range(len(cols)):
    for j in range(i+1,len(cols)):
        item_g=[cols[i],cols[j]]
        temp=data[['prefix','query','title','tag','label']].groupby(item_g,as_index=False)['label'].agg({'_'.join(item_g)+'_click':'sum','_'.join(item_g)+'count':'count'})
        temp['_'.join(item_g)+'_ctr']=temp['_'.join(item_g)+'_click']/(temp['_'.join(item_g)+'count']+3)
        train_data=pd.merge(train_data,temp,on=item_g,how='left')
print(train_data.shape)

(2099998, 51)


In [10]:
train_dd=train_data.drop(['prefix','query','title','tag','query_len','prefix_len','tag_len','title_len','label'],axis=1)

In [11]:
data=pd.concat([data,train_dd],axis=1)
print(data.shape)

(2099998, 51)


In [13]:
print(data.columns)

Index(['prefix', 'query', 'title', 'tag', 'label', 'query_len', 'prefix_len',
       'tag_len', 'title_len', 'prefix_click', 'prefix_count', 'prefix_ctr',
       'query_click', 'query_count', 'query_ctr', 'title_click', 'title_count',
       'title_ctr', 'tag_click', 'tag_count', 'tag_ctr', 'query_len_click',
       'query_len_count', 'query_len_ctr', 'prefix_len_click',
       'prefix_len_count', 'prefix_len_ctr', 'tag_len_click', 'tag_len_count',
       'tag_len_ctr', 'title_len_click', 'title_len_count', 'title_len_ctr',
       'prefix_query_click', 'prefix_querycount', 'prefix_query_ctr',
       'prefix_title_click', 'prefix_titlecount', 'prefix_title_ctr',
       'prefix_tag_click', 'prefix_tagcount', 'prefix_tag_ctr',
       'query_title_click', 'query_titlecount', 'query_title_ctr',
       'query_tag_click', 'query_tagcount', 'query_tag_ctr', 'title_tag_click',
       'title_tagcount', 'title_tag_ctr'],
      dtype='object')


In [12]:
#去掉方差为0的没有意义的数据
# v=data[data['label']!=-1].var()
# zero_fea=v[v==0].index.values.tolist()

# print(zero_fea)
# 只有['query_len_15']

# ===========Train===============

In [14]:
traindf=data[data['label']!=-1]
testdf=data[data['label']==-1]
print(traindf.shape,testdf.shape)

(2049998, 51) (50000, 51)


In [15]:
traindf.to_csv('./data/final_train1.csv',index=False)
testddf=testdf.drop(['label'],axis=1)
testddf.to_csv('./data/final_test1.csv',index=False)

In [16]:
y=traindf.pop('label').values
X=traindf.as_matrix()
testX=testddf.as_matrix()

In [40]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

In [25]:
sub=pd.DataFrame()

In [44]:
logloss=[]
losstrain=0
lossval=0
fs=0
ft=0

N=5

params={'boosting_type':'gbdt','objective':'binary','metric':'binary_logloss',
                    'num_leaves':50,'learning_rate':0.05,'feature_fraction':0.9,
                       'bagging_fraction':0.8,'bagging_freq':5,'verbose':1}

skf=StratifiedKFold(n_splits=N,random_state=1,shuffle=True)

# clf=lgb.LGBMClassifier(boosting_type='gbdt',objective='binary',metric='binary_logloss',
#                     num_leaves=500,learning_rate=0.05,feature_fraction=0.9,
#                        bagging_fraction=0.8,bagging_freq=5,max_depth=-1,
#                       verbose=1)

In [45]:
for i,(train_index,test_index) in enumerate(skf.split(X,y)):
    print("Fold=%s"%i)
    X_train,X_test,y_train,y_test=X[train_index],X[test_index],y[train_index],y[test_index]
    
    lgb_train=lgb.Dataset(X_train,y_train)
    lgb_val=lgb.Dataset(X_test,y_test,reference=lgb_train)
    
    model=lgb.train(params,lgb_train,num_boost_round=5000,valid_sets=[lgb_train,lgb_val],
                    early_stopping_rounds=50,verbose_eval=50,)
    
    trainss=model.best_score['training']['binary_logloss']
    valss=model.best_score['valid_1']['binary_logloss']
    losstrain+=trainss
    lossval+=valss
    
    pred_proba=model.predict(X_test,num_iteration=model.best_iteration)
#     print(pred_proba)
    pred_val=np.where(pred_proba>0.5,1,0)
#     print(pred_val)

    ftrain=f1_score(y_train,np.where(model.predict(X_train,num_iteration=model.best_iteration)>0.5,1,0))
    print(ftrain)
    ft+=ftrain
    
    fval=f1_score(y_test,pred_val)
    print(fval)
    fs+=fval
    
    pred_test=model.predict(testX,num_iteration=model.best_iteration)
    sub['ans_%s'%str(i)]=pred_test
print("train_set上最好的logloss成绩的平均值为：%.5f"%(losstrain/N))
print("val_set上最好的logloss成绩的平均值为：%.5f"%(lossval/N))
print("train_set上f1_score成绩的平均值为：%.5f"%(ft/N))
print("val_set上f1_score成绩的平均值为：%.5f"%(fs/N))

Fold=0
Training until validation scores don't improve for 50 rounds.
[50]	training's binary_logloss: 0.345906	valid_1's binary_logloss: 0.345435
[100]	training's binary_logloss: 0.332722	valid_1's binary_logloss: 0.332467
[150]	training's binary_logloss: 0.331001	valid_1's binary_logloss: 0.331071
[200]	training's binary_logloss: 0.33022	valid_1's binary_logloss: 0.330646
[250]	training's binary_logloss: 0.329684	valid_1's binary_logloss: 0.330553
[300]	training's binary_logloss: 0.329229	valid_1's binary_logloss: 0.33057
Early stopping, best iteration is:
[261]	training's binary_logloss: 0.329574	valid_1's binary_logloss: 0.330543
0.7914858581862936
0.7916223557382934
Fold=1
Training until validation scores don't improve for 50 rounds.
[50]	training's binary_logloss: 0.34565	valid_1's binary_logloss: 0.346337
[100]	training's binary_logloss: 0.332491	valid_1's binary_logloss: 0.333526
[150]	training's binary_logloss: 0.330706	valid_1's binary_logloss: 0.332072
[200]	training's binary_

In [46]:
print(sub)

          ans_0     ans_1     ans_2     ans_3     ans_4
0      0.014057  0.018538  0.013557  0.019247  0.013875
1      0.530977  0.539832  0.538347  0.539643  0.533055
2      0.589679  0.612567  0.585650  0.566930  0.603303
3      0.014473  0.013645  0.015079  0.012150  0.010269
4      0.075484  0.080884  0.075259  0.073760  0.078914
5      0.006348  0.006490  0.008296  0.009014  0.007018
6      0.823319  0.816836  0.830985  0.826371  0.816116
7      0.011301  0.010516  0.010688  0.010285  0.010365
8      0.480792  0.476122  0.495312  0.466013  0.478915
9      0.037361  0.038376  0.040930  0.036188  0.036807
10     0.006291  0.004714  0.004606  0.006447  0.006690
11     0.505602  0.485726  0.506491  0.491682  0.498171
12     0.004049  0.011405  0.004420  0.003196  0.003407
13     0.007704  0.003913  0.006445  0.003360  0.005751
14     0.284699  0.277025  0.286368  0.280425  0.284581
15     0.084564  0.086085  0.084410  0.087969  0.089930
16     0.630350  0.607306  0.619467  0.623222  0

In [47]:
sub['label']=0
for i in range(N):
    sub['label']+=sub['ans_%s'%str(i)]
sub['label']=sub['label']/N
sub['label']=sub['label'].apply(lambda x:np.where(x>0.5,1,0))

In [49]:
sub.head()

,ans_0,ans_1,ans_2,ans_3,ans_4,label
0,0.014057,0.018538,0.013557,0.019247,0.013875,0
1,0.530977,0.539832,0.538347,0.539643,0.533055,1
2,0.589679,0.612567,0.585650,0.566930,0.603303,1
3,0.014473,0.013645,0.015079,0.012150,0.010269,0
4,0.075484,0.080884,0.075259,0.073760,0.078914,0


In [55]:
sub['label'].to_csv('./data/BaselineV1_mean.csv',header=False,index=False)

In [56]:
print(sub.label.value_counts())

0    38265
1    11735
Name: label, dtype: int64


In [57]:
sub['sum']=0
for i in range(N):
    sub['ans_%s'%str(i)]=sub['ans_%s'%str(i)].apply(lambda x:np.where(x>0.5,1,0))
    sub['sum']+=sub['ans_%s'%str(i)]
sub['sum']=sub['sum'].apply(lambda x:np.where(x>2.5,1,0))
print(sub['sum'].value_counts())

0    38262
1    11738
Name: sum, dtype: int64


In [58]:
sub['sum'].to_csv('./data/BaselineV1_vote.csv',header=False,index=False)

# 成绩为0.6231，非常的差！过拟合现象非常严重！

<font color=red>2018/11/1<br>过拟合现象之所以严重，与XXY版本对比，可以发现，很大的原因是是因为feature之间相互作用而导致的！